## A battle between Manhattan and Brooklyn

Import the needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.33 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  23.44 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  32.42 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  17.39 MB/s
vincent-0.4.4- 100% |###################

### Datasets

The data that will be used in this solution is from a json file available at https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json. The json file has the data related to all neighborhoods in the city of NewYork.

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

Creating dataframes with the neighborhood data of Newyork city.

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In this solution we will focus on the Neighborhoods in Manhattan and Brooklyn. So, we will create separate dataframes containing the neighborhoods of Manhattan and Brooklyn.

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

### Visualizing the neighborhoods of Manhattan on a map

In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [9]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Visualizing the neighborhoods of Brooklyn

In [10]:
brooklyn_address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(brooklyn_address)
brooklyn_latitude = location.latitude
brooklyn_longitude = location.longitude
print('The geograpical coordinate of Brroklyn are {}, {}.'.format(brooklyn_latitude, brooklyn_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Brroklyn are 40.6501038, -73.9495823.


In [11]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[brooklyn_latitude, brooklyn_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### Explore the neighborhoods of Manhattan using Foursquare API

In [12]:
CLIENT_ID = 'Y5D4KU0ZLWXFPUBBDF4DTAC2HIOMHMK5QMRHCWRPTIU5HMDS' # your Foursquare ID
CLIENT_SECRET = 'N3CEZBG1V1TUS55RU1U0M2UDZTKQWGH1VV1DNKXCLXYK2CMJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y5D4KU0ZLWXFPUBBDF4DTAC2HIOMHMK5QMRHCWRPTIU5HMDS
CLIENT_SECRET:N3CEZBG1V1TUS55RU1U0M2UDZTKQWGH1VV1DNKXCLXYK2CMJ


In [13]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)   
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [16]:
print('There are {} venues in Brooklyn.'.format(manhattan_venues.shape[0]))

There are 3308 venues in Brooklyn.


### Explore the neighborhoods of Brooklyn using Foursquare API.

In [21]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [22]:
print('There are {} venues in Brooklyn.'.format(brooklyn_venues.shape[0]))

There are 2810 venues in Brooklyn.


Filter the avenues in both Manhattan and Brooklyn to get the restaurants data.

In [23]:
def asianrestaurantcount(row):
    
    neighborhood=row['Neighborhood']
    restaurants_neighborhood=restaurants_df[restaurants_df['Neighborhood']==neighborhood]
    asianrestaurants_neighborhood=restaurants_neighborhood[restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    
    return asianrestaurants_neighborhood.shape[0]

In [24]:
def brooklyn_asianrestaurantcount(row):
    
    neighborhood=row['Neighborhood']
    brooklyn_restaurants_neighborhood=brooklyn_restaurants_df[brooklyn_restaurants_df['Neighborhood']==neighborhood]
    brooklyn_asianrestaurants_neighborhood=brooklyn_restaurants_neighborhood[brooklyn_restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    
    return brooklyn_asianrestaurants_neighborhood.shape[0]

In [25]:
restaurants_df=manhattan_venues[manhattan_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
restaurants=restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Manhattan is {}'.format(restaurants['Restaurant count'].sum()))

Total number of restarunats in Manhattan is 940


In [26]:
brooklyn_restaurants_df=brooklyn_venues[brooklyn_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
brooklyn_restaurants=brooklyn_restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Restaurant count'].sum()))

Total number of restarunats in Brooklyn is 662


Manhattan has more number of restaurants than Brooklyn. Now, lets further filter the restaurants to find the number of Asian Restaurants.

In [27]:
restaurants['Asian Restaurant count']=restaurants.apply(asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Manhattan is {}'.format(restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Manhattan is 15


In [28]:
brooklyn_restaurants['Asian Restaurant count']=brooklyn_restaurants.apply(brooklyn_asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Brooklyn is 17


Though Manhattan has more restaurants than Brooklyn, it has fewer Asian restaurants than Brooklyn. So, Manhattan appears to be better option to open a Asian restaurant.

Let's see the count of restaurants and Asian restaurants in each of the neighborhoods of Manhattan.

Count of restaurants and Asian restaurants in Manhattan grouped by Neighborhood:

In [29]:
restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
12,Greenwich Village,44,0
4,Chinatown,41,2
8,East Village,40,0
36,Upper West Side,38,1
34,Turtle Bay,38,1
38,West Village,37,0
27,Noho,34,1
26,Murray Hill,33,0
24,Midtown South,32,0
10,Flatiron,31,0


From the above summary, Greenwich Village and Chinatown have the most number of restaurant among the neighborhoods. In general, more number of restaurants implies that there is good demand for restaurants in that locality. Lets further study the neighborhoods of Greenwich Village and Chinatown to zero down on the best location for an Asian Restaurant.

In [30]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
print(manhattan_onehot.shape)
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
print(manhattan_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

(3308, 331)
(3308, 332)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social C

In [31]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

Let's study each neighborhood in Manhattan to know the top 5 most common venues in that neighborhood.

In [32]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0    Coffee Shop  0.08
1           Park  0.07
2          Hotel  0.05
3  Women's Store  0.03
4      Wine Shop  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1     Coffee Shop  0.05
2  Cosmetics Shop  0.05
3            Café  0.04
4             Spa  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1    Seafood Restaurant  0.04
2   American Restaurant  0.04
3    Chinese Restaurant  0.04
4  Gym / Fitness Center  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1        Bubble Tea Shop  0.06
2  Vietnamese Restaurant  0.04
3     Dim Sum Restaurant  0.04
4    American Restaurant  0.04


----Civic Center----
                  venue  freq
0  Gym / Fitness C

We can see that both Greenwich Village and Chinatown have restaurants as the 4 out of 5 most common venues in the neighborhood. But, the restaurants in Chinatown appear to be those serving cuisines which overlap with Asian cuisine. Whereas, the restaurants in Greenwich Village as more spread out in terms of the cuisine they offer.

Further, Chitatown already has 2 Asian restaurants whereas Greenwich Village doesn't have any. So, an Asian restaurant would be a good addition to the restaurants available in it.

Let's study further to see if we can find any reason to support our inference that Greenwich Village is a good location to open an Asian restaurant.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Wine Shop,Women's Store,Gym,Market,Burger Joint,Memorial Site
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,French Restaurant,Bar,Gym,Japanese Restaurant
2,Central Harlem,African Restaurant,French Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Cosmetics Shop,Tapas Restaurant,Park,Beer Bar
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,American Restaurant,Bakery,Hotel,Art Gallery,Theater,Seafood Restaurant
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bakery,Hotpot Restaurant,Noodle House
5,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,French Restaurant,Coffee Shop,Sporting Goods Shop,Gym,Spa,Sandwich Place,Cocktail Bar
6,Clinton,Theater,Coffee Shop,Gym / Fitness Center,American Restaurant,Italian Restaurant,Wine Shop,Spa,Hotel,Gym,Lounge
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Convenience Store,Café,Taco Place,Street Art,Dance Studio
8,East Village,Bar,Ice Cream Shop,Wine Bar,Cocktail Bar,Mexican Restaurant,Pizza Place,Chinese Restaurant,Ramen Restaurant,Speakeasy,Café
9,Financial District,Coffee Shop,Hotel,Wine Shop,Bar,Gym,Steakhouse,Food Truck,Italian Restaurant,Pizza Place,Falafel Restaurant


In [35]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 2, 1, 2, 1, 0, 1, 1], dtype=int32)

In [36]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bubble Tea Shop,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bakery,Hotpot Restaurant,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Wine Shop,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Pizza Place,Café,Bakery,Lounge,American Restaurant,Deli / Bodega,Restaurant,Park,Pharmacy
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Convenience Store,Café,Taco Place,Street Art,Dance Studio
13,Lincoln Square,Gym / Fitness Center,Theater,Concert Hall,Plaza,Italian Restaurant,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue
15,Midtown,Hotel,Steakhouse,Food Truck,Coffee Shop,Theater,Cocktail Bar,Bakery,Clothing Store,Sporting Goods Shop,Spa
20,Lower East Side,Coffee Shop,Café,Ramen Restaurant,Chinese Restaurant,Bakery,Art Gallery,Cocktail Bar,Sandwich Place,Latin American Restaurant,Shoe Store
25,Manhattan Valley,Coffee Shop,Pizza Place,Thai Restaurant,Szechuan Restaurant,Playground,French Restaurant,Bar,Mexican Restaurant,Yoga Studio,Café
28,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Wine Shop,Women's Store,Gym,Market,Burger Joint,Memorial Site
33,Midtown South,Korean Restaurant,Cosmetics Shop,Coffee Shop,Japanese Restaurant,Hotel Bar,Boutique,Bakery,Gym / Fitness Center,Hotel,Italian Restaurant
37,Stuyvesant Town,Bar,Park,Playground,Gas Station,Farmers Market,Baseball Field,Beer Garden,Harbor / Marina,Cocktail Bar,Coffee Shop


The neighborhoods in cluster 0 predominantly have eatery joints as the top 10 common venues. Greenwich Village is also in this cluster, this further strengths our inference that it is a good option to open an Asian restaurant.

The added advantage for Greenwich village is it is in close proximity of affluent neighborhoods such as Soho and Tribeca. So, it is quite possible that people of these affluent neighborhoods would visit a restaurant that serves a new cuisine in a nearby locality.

Based on the above findings, we can conclude that Greenwich village is the best neighborhood among the neighborhoods of Manhattan and Brooklyn to open an Asian Restaurant.

In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
1,Chinatown,Chinese Restaurant,Bubble Tea Shop,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bakery,Hotpot Restaurant,Noodle House
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Wine Shop,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Park
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant
6,Central Harlem,African Restaurant,French Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Cosmetics Shop,Tapas Restaurant,Park,Beer Bar
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Coffee Shop,Bakery,Gym / Fitness Center,Hotel,French Restaurant,Boutique,Juice Bar
9,Yorkville,Coffee Shop,Gym,Bar,Italian Restaurant,Pizza Place,Deli / Bodega,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Bagel Shop
11,Roosevelt Island,Park,Sandwich Place,Rental Car Location,Dog Run,Bus Line,Bus Station,Supermarket,Café,Farmers Market,Outdoors & Recreation
14,Clinton,Theater,Coffee Shop,Gym / Fitness Center,American Restaurant,Italian Restaurant,Wine Shop,Spa,Hotel,Gym,Lounge
19,East Village,Bar,Ice Cream Shop,Wine Bar,Cocktail Bar,Mexican Restaurant,Pizza Place,Chinese Restaurant,Ramen Restaurant,Speakeasy,Café


In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Mexican Restaurant,Pizza Place,Café,Bakery,Lounge,American Restaurant,Deli / Bodega,Restaurant,Park,Pharmacy
5,Manhattanville,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Sushi Restaurant,Supermarket,Burger Joint,Bus Station,Spanish Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Sporting Goods Shop,Gym,Burger Joint,Art Gallery,Bakery
12,Upper West Side,Italian Restaurant,Bar,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,Burger Joint,Indian Restaurant,Wine Bar,French Restaurant,Seafood Restaurant
16,Murray Hill,Hotel,Spa,Japanese Restaurant,Bar,Italian Restaurant,French Restaurant,Gym / Fitness Center,Gym,Sandwich Place,Salon / Barbershop
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,American Restaurant,Bakery,Hotel,Art Gallery,Theater,Seafood Restaurant
18,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Clothing Store,Indian Restaurant,Chinese Restaurant,Seafood Restaurant,Café,Yoga Studio,Gourmet Shop
24,West Village,Italian Restaurant,New American Restaurant,Cosmetics Shop,Wine Bar,Jazz Club,Gastropub,French Restaurant,Park,American Restaurant,Bakery
27,Gramercy,Italian Restaurant,Coffee Shop,Wine Shop,Bagel Shop,Restaurant,Cocktail Bar,Thrift / Vintage Store,American Restaurant,Grocery Store,Pizza Place
29,Financial District,Coffee Shop,Hotel,Wine Shop,Bar,Gym,Steakhouse,Food Truck,Italian Restaurant,Pizza Place,Falafel Restaurant


In [41]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,French Restaurant,Bar,Gym,Japanese Restaurant


In [42]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Little Italy,Bakery,Café,Yoga Studio,Ice Cream Shop,Sandwich Place,Seafood Restaurant,Cocktail Bar,Salon / Barbershop,Bubble Tea Shop,Japanese Restaurant


### Analysis of Brooklyn neighborhoods, for reference only.

In [43]:
brooklyn_restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
55,South Side,35,2
14,Clinton Hill,35,0
1,Bay Ridge,30,1
43,North Side,27,1
48,Prospect Heights,27,0
27,Fort Greene,26,0
15,Cobble Hill,26,0
12,Carroll Gardens,24,1
47,Park Slope,23,0
8,Brooklyn Heights,23,2


In [44]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
print(brooklyn_onehot.shape)
# add neighborhood column back to dataframe
brooklyn_onehot['Neighbourhood'] = brooklyn_venues['Neighborhood'] 
print(brooklyn_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]
#brooklyn_onehot.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
brooklyn_onehot.head()

(2810, 286)
(2810, 287)


,Neighbourhood,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Resta

In [45]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighbourhood').mean().reset_index()
brooklyn_grouped

,Neighbourhood,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Resta

In [46]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0            Pharmacy  0.06
1          Donut Shop  0.06
2  Italian Restaurant  0.04
3        Dessert Shop  0.04
4          Kids Store  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.06
1          Pizza Place  0.06
2                  Spa  0.05
3  American Restaurant  0.03
4     Greek Restaurant  0.03


----Bedford Stuyvesant----
         venue  freq
0  Coffee Shop  0.10
1     Bus Stop  0.07
2  Pizza Place  0.07
3         Café  0.07
4          Bar  0.07


----Bensonhurst----
                    venue  freq
0          Ice Cream Shop  0.07
1                  Bakery  0.07
2        Sushi Restaurant  0.07
3  Shabu-Shabu Restaurant  0.04
4       Hotpot Restaurant  0.04


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1          Donut Shop  0.17
2      Baseball Field  0.17
3          Playground  0.17
4  Athletics & Sports  0.17


----Boerum Hill----
               venue  freq
0        C

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brooklyn_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighbourhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Donut Shop,Pharmacy,Italian Restaurant,Chinese Restaurant,Dessert Shop,Kids Store,Pizza Place,Mobile Phone Shop,Sushi Restaurant,Asian Restaurant
1,Bay Ridge,Italian Restaurant,Pizza Place,Spa,Bagel Shop,American Restaurant,Chinese Restaurant,Grocery Store,Bar,Greek Restaurant,Seafood Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Bus Stop,Pizza Place,Bar,BBQ Joint,Fruit & Vegetable Store,Boutique,New American Restaurant,Gourmet Shop
3,Bensonhurst,Ice Cream Shop,Sushi Restaurant,Bakery,Bagel Shop,Shabu-Shabu Restaurant,Spa,Bridal Shop,Supermarket,Chinese Restaurant,Liquor Store
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Baseball Field,Playground,Donut Shop,Food & Drink Shop,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop
5,Boerum Hill,Coffee Shop,Bar,Spa,Dance Studio,French Restaurant,Martial Arts Dojo,Boutique,Grocery Store,Furniture / Home Store,Cosmetics Shop
6,Borough Park,Pizza Place,Bank,Café,Pharmacy,Bakery,Fast Food Restaurant,Coffee Shop,Farmers Market,Restaurant,Kosher Restaurant
7,Brighton Beach,Beach,Restaurant,Eastern European Restaurant,Gourmet Shop,Neighborhood,Bank,Mobile Phone Shop,Fast Food Restaurant,Sushi Restaurant,Russian Restaurant
8,Broadway Junction,Diner,Donut Shop,Bus Station,Fried Chicken Joint,Caribbean Restaurant,Sandwich Place,Moving Target,Burger Joint,Deli / Bodega,Ice Cream Shop
9,Brooklyn Heights,Yoga Studio,Italian Restaurant,Park,Deli / Bodega,Coffee Shop,Pizza Place,Gym,Pet Store,Juice Bar,Mexican Restaurant


In [48]:
# set number of clusters
brooklyn_kclusters =8

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighbourhood', 1)

# run k-means clustering
brooklyn_kmeans = KMeans(n_clusters=brooklyn_kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
brooklyn_kmeans.labels_[0:10]

array([0, 5, 5, 5, 4, 5, 0, 0, 0, 5], dtype=int32)

In [49]:
brooklyn_merged = brooklyn_data

# add clustering labels
brooklyn_merged['Cluster Labels'] = brooklyn_kmeans.labels_

brooklyn_merged = brooklyn_merged.join(brooklyn_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Italian Restaurant,Pizza Place,Spa,Bagel Shop,American Restaurant,Chinese Restaurant,Grocery Store,Bar,Greek Restaurant,Seafood Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,5,Ice Cream Shop,Sushi Restaurant,Bakery,Bagel Shop,Shabu-Shabu Restaurant,Spa,Bridal Shop,Supermarket,Chinese Restaurant,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,5,Bank,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Gym,Mobile Phone Shop,Pharmacy,Deli / Bodega,Record Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,5,Bar,Pizza Place,Coffee Shop,Café,Cocktail Bar,Yoga Studio,Record Shop,Mexican Restaurant,Bakery,Italian Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,4,Lounge,Bus Station,Pizza Place,Martial Arts Dojo,Bakery,Deli / Bodega,Italian Restaurant,Sporting Goods Shop,Men's Store,Doctor's Office
